In [0]:
import torch
import numpy as np
import pandas as pd
from sklearn import datasets
import torch.nn.functional as F
import torch.nn as nn 
from torch import optim
from sklearn.model_selection import train_test_split

In [2]:
breast_cancer=datasets.load_wine()
X=breast_cancer.data
Y=breast_cancer.target
print(X.shape,Y.shape)

(178, 13) (178,)


In [3]:
from sklearn import preprocessing
mm_scaler = preprocessing.MinMaxScaler()
X = mm_scaler.fit_transform(X)
print(X)

[[0.84210526 0.1916996  0.57219251 ... 0.45528455 0.97069597 0.56134094]
 [0.57105263 0.2055336  0.4171123  ... 0.46341463 0.78021978 0.55064194]
 [0.56052632 0.3201581  0.70053476 ... 0.44715447 0.6959707  0.64693295]
 ...
 [0.58947368 0.69960474 0.48128342 ... 0.08943089 0.10622711 0.39728959]
 [0.56315789 0.36561265 0.54010695 ... 0.09756098 0.12820513 0.40085592]
 [0.81578947 0.66403162 0.73796791 ... 0.10569106 0.12087912 0.20114123]]


In [4]:
#splitting data into  50% train and 50% test
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,stratify=Y,random_state=0,test_size=0.50)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(89, 13) (89, 13) (89,) (89,)


In [5]:
df = pd.DataFrame(data=X_train)
print(df.head())
print(Y_train)

         0         1         2   ...        10        11        12
0  0.152632  0.120553  0.716578  ...  0.390244  0.728938  0.286733
1  0.578947  0.505929  0.491979  ...  0.065041  0.087912  0.283167
2  0.478947  0.500000  0.652406  ...  0.317073  0.307692  0.208274
3  0.665789  0.191700  0.508021  ...  0.382114  0.772894  0.368759
4  0.621053  0.203557  0.673797  ...  0.357724  0.714286  0.654066

[5 rows x 13 columns]
[1 2 2 0 0 0 1 0 2 1 0 1 2 1 1 0 1 1 1 1 0 1 0 0 1 1 1 1 0 2 2 2 2 0 0 0 0
 1 0 0 2 2 2 1 0 0 0 1 1 0 1 1 1 2 0 2 2 1 2 2 0 1 2 1 1 0 1 1 0 1 2 2 2 0
 1 2 1 1 1 1 0 0 1 0 0 1 2 2 2]


In [6]:
#covert all data into tensors
X_train,X_test,Y_train,Y_test=map(torch.tensor ,(X_train,X_test,Y_train,Y_test))
print(X_train.dtype, X_test.dtype, Y_train.dtype, Y_test.dtype)
X_train.shape
print(Y_train)

torch.float64 torch.float64 torch.int64 torch.int64
tensor([1, 2, 2, 0, 0, 0, 1, 0, 2, 1, 0, 1, 2, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2, 2, 1, 0, 0, 0, 1,
        1, 0, 1, 1, 1, 2, 0, 2, 2, 1, 2, 2, 0, 1, 2, 1, 1, 0, 1, 1, 0, 1, 2, 2,
        2, 0, 1, 2, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 2, 2, 2])


In [0]:
#convert all tensors to float to avoid in training
X_train=X_train.float()
X_test=X_test.float()
Y_train=Y_train.long()
Y_test=Y_test.long()

In [0]:
#constructing class to built neural network
class Classfication(nn.Module):
  def __init__(self):
     super().__init__()
     torch.manual_seed(0)
     self.net=nn.Sequential(
         nn.Linear(13,50),
         nn.ReLU(),
         nn.Linear(50,100),
         nn.ReLU(),
         nn.Linear(100,50),
         nn.ReLU(),
         nn.Linear(50,3),
         nn.Softmax()
     )
  def forward(self,x):
    return self.net(x)


**TRAIN USING CPU**

In [0]:
#model has acuraccy of 100% test accuracy in 50-50 train test split with epoch 100
def fit(x,y,model,opt,loss_fn,epochs=100):
  for epoch in range(epochs):
    
    loss=loss_fn(model(x),y)
    
    loss.backward()
    
    opt.step()
    opt.zero_grad()

  return loss.item()

In [10]:
model=Classfication()
loss_fn=F.cross_entropy
opt=optim.Adam(model.parameters(),lr=0.01)
fit(X_train,Y_train, model, opt, loss_fn)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.5514489412307739

In [11]:
y_hat=model(X_test)
y_np=y_hat.detach().numpy()
y_model=np.argmax(y_np,axis=1)
print(y_model)

[0 2 0 0 1 1 2 1 2 1 1 1 1 1 1 1 2 2 0 1 2 0 2 0 1 2 1 0 0 0 0 0 0 2 2 1 1
 1 1 1 2 0 1 2 1 0 0 0 2 1 0 2 2 0 0 1 1 0 0 1 2 2 2 1 0 0 0 0 1 0 0 2 2 0
 1 2 1 1 0 1 0 1 2 1 2 1 2 0 1]


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [12]:
y_test=Y_test.detach().numpy()
print(y_test)

[0 2 0 0 1 1 2 1 2 1 1 1 1 1 1 1 2 2 0 1 2 0 2 0 1 2 1 0 0 0 0 0 0 2 2 1 1
 1 1 1 2 0 1 2 1 0 1 0 2 0 0 2 2 0 0 1 1 0 0 1 2 2 2 1 0 0 0 0 1 0 1 2 2 0
 1 2 1 1 0 1 0 1 2 1 2 1 2 0 1]


In [13]:
count=0
for y_pred,y_true in zip(y_model,y_test):
  if(y_pred==y_true):
      count=count + 1
accuracy=(count/y_model.shape[0])*100
print("your model accuracy:",accuracy)

your model accuracy: 96.62921348314607


**TRAIN USING GPU**

In [0]:
def fit_gpu(x,y,model,opt,loss_fn,epochs=100):
  for epoch in range(epochs):
    loss=loss_fn(model(x),y)
    
    loss.backward()
    
    opt.step()
    opt.zero_grad()

  return loss.item()

In [0]:
device=torch.device('cuda')

In [20]:
model=Classfication()
model=model.to(device)
loss_fn=F.cross_entropy
opt=optim.Adam(model.parameters(),lr=0.01)
X_train,Y_train=X_train.to(device),Y_train.to(device)
fit(X_train,Y_train, model, opt, loss_fn)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.5514488816261292

In [22]:
y_hat=model(X_test.to(device))
y_np=y_hat.detach().cpu().numpy()
y_model=np.argmax(y_np,axis=1)
#print(y_model)
y_test=Y_test.numpy()
#print(y_test)
count=0
for y_pred,y_true in zip(y_model,y_test):
  if(y_pred==y_true):
      count=count + 1
accuracy=(count/y_model.shape[0])*100
print("your model accuracy:",accuracy)

your model accuracy: 96.62921348314607


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:100: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
